In [28]:
def bytes_info(size): #выводит размер файла в байтах, килобайтах и мегабайтах
    print('Размер файла:',  size, 'байт или ', round(size/1024, 3), 'Кб или ', round(size/1024**2, 3), 'Мб')

def compare_size(old, new): #выводит процент, который составляет новый размер файла от старого
    print('Новый размер файла составляет ', round(new/old, 3)*100, '% от старого')

In [29]:
# загружаем текст и вычисляем его исходный размер
with open('../text_for_decap.txt', 'r', encoding='ascii') as f:
        text = f.read()

main_size=len(bytearray(text, 'ascii'))
bytes_info(main_size)

text1=text
text2=text.lower()
with open('text_decape.txt', 'w', encoding='ascii') as f:
        f.write(text2)
text3=text1[::-1]
with open('text_for_decap_reverse.txt', 'w', encoding='ascii') as f:
        f.write(text3)
text4=text2[::-1]
with open('text_decap_reverse.txt', 'w', encoding='ascii') as f:
        f.write(text4)


Размер файла: 3171529 байт или  3097.196 Кб или  3.025 Мб


In [30]:
import math
from collections import defaultdict
from typing import Union, List

SymbolType = Union[str, int]

class PPMModel:
    def __init__(self, max_order: int, escape_method: str):
        self.max_order = max_order
        self.escape_method = escape_method
        
        self.context_trees = [defaultdict(dict) for _ in range(max_order + 2)]
        self.total_symbols = 0
        self.unique_symbols = set()
        
        self.escape_stats = {
            'deterministic': defaultdict(lambda: {'seen': 0, 'escapes': 0}),
            'max_order': defaultdict(lambda: {'seen': 0, 'escapes': 0}),
            'other': defaultdict(lambda: {'seen': 0, 'escapes': 0})
        }
    
    def update_model(self, context: List[SymbolType], symbol: SymbolType):
        self.total_symbols += 1
        self.unique_symbols.add(symbol)
        
        for order in range(min(len(context), self.max_order) + 1):
            current_context = tuple(context[-order:]) if order > 0 else tuple()
            
            if current_context not in self.context_trees[order]:
                self.context_trees[order][current_context] = {
                    'count': 0,
                    'symbols': defaultdict(int),
                    'total': 0
                }
            
            ctx = self.context_trees[order][current_context]
            ctx['symbols'][symbol] += 1
            ctx['count'] += 1
            ctx['total'] += 1
    
    def get_escape_probability(self, context: tuple, order: int) -> float:
        if context not in self.context_trees[order]:
            return 1.0
        
        ctx = self.context_trees[order][context]
        m = len(ctx['symbols'])
        m1 = sum(1 for value in ctx['symbols'].values() if value == 1)
        c = ctx['count']
        
        if self.escape_method == 'A':
            return 1.0 / (c + 1)
        elif self.escape_method == 'B':
            return m / c
        elif self.escape_method == 'C':
            return m / (c + m)
        elif self.escape_method == 'D':
            return m / (2 * c)
        elif self.escape_method == 'X':
            return m1 / c
        else:
            return m / (c + m)
    
    def get_symbol_probs(self, context: tuple, order: int, masked_symbols: set) -> dict:
        if context not in self.context_trees[order]:
            return {}
        
        ctx = self.context_trees[order][context]
        available_symbols = {s: cnt for s, cnt in ctx['symbols'].items() 
                           if s not in masked_symbols}
        
        if not available_symbols:
            return {}
        
        total = sum(available_symbols.values())
        p_escape = self.get_escape_probability(context, order)
        scale = 1 - p_escape
        
        probs = {s: (cnt / total) * scale for s, cnt in available_symbols.items()}
        return probs
    
    def encode_symbol(self, context_so_far: List[SymbolType], symbol: SymbolType, encoder):
        encoded = False
        escaped = False
        masked_symbols = set()
        
        for order in range(min(len(context_so_far), self.max_order), -1, -1):
            context = tuple(context_so_far[-order:]) if order > 0 else tuple()
            
            symbol_probs = self.get_symbol_probs(context, order, masked_symbols)
            
            if symbol in symbol_probs:
                # Нормализуем вероятности
                total_prob = sum(symbol_probs.values())
                if total_prob <= 0:
                    continue
                    
                normalized_probs = {s: p/total_prob for s, p in symbol_probs.items()}
                
                # Кодируем символ
                encoder.encode_symbol(normalized_probs[symbol], symbol, list(normalized_probs.keys()))
                encoded = True
                escaped = False
                break
            else:
                # Кодируем escape-символ
                p_escape = self.get_escape_probability(context, order)
                encoder.encode_escape(p_escape)
                if context in self.context_trees[order]:
                    masked_symbols.update(self.context_trees[order][context]['symbols'].keys())
                escaped = True
        
        if not encoded:
            if len(self.unique_symbols) == 0:
                # Кодируем первый символ
                encoder.encode_first_symbol(symbol)
            else:
                # Кодируем новый символ
                p_new = 1.0 / (len(self.unique_symbols) + 1)
                encoder.encode_new_symbol(p_new, symbol, list(self.unique_symbols))
        
        self.update_model(context_so_far, symbol)

class ArithmeticEncoder:
    def __init__(self):
        self.low = 0
        self.high = 0xFFFFFFFF
        self.pending_bits = 0
        self.output = bytearray()
        self.buffer = 0
        self.buffer_pos = 0
        self.first_symbol = None
    
    def encode_first_symbol(self, symbol: SymbolType):
        if isinstance(symbol, str):
            self.first_symbol = symbol.encode('ascii')
        else:
            self.first_symbol = symbol.to_bytes(4, byteorder='big', signed=True)
    
    def encode_new_symbol(self, probability: float, symbol: SymbolType, symbol_list: List[SymbolType]):
        range_size = self.high - self.low + 1
        symbol_index = len(symbol_list)
        
        self.high = self.low + (range_size * (symbol_index + 1)) // (len(symbol_list) + 1) - 1
        self.low = self.low + (range_size * symbol_index) // (len(symbol_list) + 1)
        
        self._normalize()
        
        # Записываем сам символ
        if isinstance(symbol, str):
            encoded_symbol = symbol.encode('ascii')
        else:
            encoded_symbol = symbol.to_bytes(4, byteorder='big', signed=True)
        
        for byte in encoded_symbol:
            self._output_byte(byte)
    
    def encode_symbol(self, probability: float, symbol: SymbolType, symbol_list: List[SymbolType]):
        range_size = self.high - self.low + 1
        symbol_index = symbol_list.index(symbol)
        
        self.high = self.low + (range_size * (symbol_index + 1)) // len(symbol_list) - 1
        self.low = self.low + (range_size * symbol_index) // len(symbol_list)
        
        self._normalize()
    
    def encode_escape(self, probability: float):
        range_size = self.high - self.low + 1
        self.high = self.low + int(range_size * probability) - 1
        self._normalize()
    
    def _normalize(self):
        while True:
            if (self.high ^ self.low) & 0x80000000 == 0:
                bit = (self.high >> 31) & 1
                self._output_bit(bit)
                
                self.low = (self.low << 1) & 0xFFFFFFFF
                self.high = ((self.high << 1) | 1) & 0xFFFFFFFF
            elif (self.low & 0x40000000) and not (self.high & 0x40000000):
                self.pending_bits += 1
                self.low = (self.low << 1) & 0x7FFFFFFF
                self.high = ((self.high << 1) | 0x80000001) & 0xFFFFFFFF
            else:
                break
    
    def _output_bit(self, bit: int):
        self.buffer = (self.buffer << 1) | bit
        self.buffer_pos += 1
        
        if self.buffer_pos == 8:
            self.output.append(self.buffer)
            self.buffer = 0
            self.buffer_pos = 0
        
        pending = self.pending_bits
        self.pending_bits = 0
        for _ in range(pending):
            self._output_bit(1 - bit)
    
    def _output_byte(self, byte: int):
        for i in reversed(range(8)):
            self._output_bit((byte >> i) & 1)
    
    def get_encoded_data(self) -> bytes:
        self.pending_bits += 1
        bit = (self.low >> 30) & 1
        self._output_bit(bit)
        
        while self.buffer_pos > 0:
            self._output_bit(0)
        
        if self.first_symbol:
            return bytes([len(self.first_symbol)]) + self.first_symbol + bytes(self.output)
        return bytes(self.output)

def ppm_compress(data: Union[str, List[int]], max_order: int, escape_method: str) -> bytes:
    model = PPMModel(max_order, escape_method)
    encoder = ArithmeticEncoder()
    
    context_so_far = []
    
    if isinstance(data, str):
        symbols = list(data)
    else:
        symbols = data
    
    for symbol in symbols:
        model.encode_symbol(context_so_far, symbol, encoder)
        context_so_far.append(symbol)
    
    return encoder.get_encoded_data()

# Пример использования
if __name__ == "__main__":
    # Пример со строкой
    text='aaabbbcacbaca'
    text=text1[:1000]
    print("Исходный текст:", text[:100], '...')
    compressed_text = ppm_compress(text, max_order=3, escape_method='X')
    print(f"Сжатые данные X (текст, размер: {len(compressed_text)} байт)")

    compressed_text = ppm_compress(text, max_order=3, escape_method='A')
    print(f"Сжатые данные A (текст, размер: {len(compressed_text)} байт)")

    compressed_text = ppm_compress(text, max_order=3, escape_method='B')
    print(f"Сжатые данные B (текст, размер: {len(compressed_text)} байт)")

    compressed_text = ppm_compress(text, max_order=3, escape_method='C')
    print(f"Сжатые данные C (текст, размер: {len(compressed_text)} байт)")

    compressed_text = ppm_compress(text, max_order=3, escape_method='D')
    print(f"Сжатые данные D (текст, размер: {len(compressed_text)} байт)")
    
    # # Пример с числами
    numbers = [1, 2, 3, 2, 1, 100, 101, 100, 3, 2, 1]
    print("\nИсходные числа:", numbers)
    compressed_numbers = ppm_compress(numbers, max_order=3, escape_method='X')
    print(f"Сжатые данные (числа, размер: {len(compressed_numbers)} байт)")

Исходный текст: VOLUME I FANTINE BOOK FIRST - A JUST MAN CHAPTER I - M. MYRIEL In 1815, M. Charles-Francois-Bienvenu ...
Сжатые данные X (текст, размер: 84 байт)
Сжатые данные A (текст, размер: 650 байт)
Сжатые данные B (текст, размер: 519 байт)
Сжатые данные C (текст, размер: 537 байт)
Сжатые данные D (текст, размер: 556 байт)

Исходные числа: [1, 2, 3, 2, 1, 100, 101, 100, 3, 2, 1]
Сжатые данные (числа, размер: 28 байт)


In [31]:
from collections import defaultdict
from typing import List, Dict, Tuple, Union
import math

SymbolType = Union[str, int]

class PPMModelX:
    def __init__(self, max_order: int):
        self.max_order = max_order
        self.context_trees = [defaultdict(lambda: {
            'count': 0,
            'symbols': defaultdict(int),
            'm1': 0  # Количество символов, встречающихся ровно 1 раз
        }) for _ in range(max_order + 1)]
        self.unique_symbols = set()

    def update_model(self, context: List[SymbolType], symbol: SymbolType):
        self.unique_symbols.add(symbol)
        
        for order in range(min(len(context), self.max_order) + 1):
            current_context = tuple(context[-order:]) if order > 0 else tuple()
            ctx = self.context_trees[order][current_context]
            
            # Обновляем m1
            if ctx['symbols'][symbol] == 0:
                ctx['m1'] += 1
            elif ctx['symbols'][symbol] == 1:
                ctx['m1'] -= 1
            
            ctx['symbols'][symbol] += 1
            ctx['count'] += 1

    def get_escape_probability(self, context: Tuple) -> float:
        order = len(context)
        if context not in self.context_trees[order]:
            return 1.0
            
        ctx = self.context_trees[order][context]
        if ctx['count'] == 0:
            return 1.0
            
        # Метод X: m1/c с ограничением диапазона
        return min(0.99, max(0.01, ctx['m1'] / ctx['count']))

class ArithmeticEncoder1:
    def __init__(self):
        self.low = 0
        self.high = 0xFFFFFFFF
        self.pending_bits = 0
        self.buffer = 0
        self.buffer_pos = 0
        self.output = bytearray()

    def encode_symbol(self, symbol: SymbolType, symbol_list: List[SymbolType]):
        range_size = self.high - self.low + 1
        symbol_index = symbol_list.index(symbol)
        total_symbols = len(symbol_list)
        
        self.high = self.low + int(range_size * (symbol_index + 1) / total_symbols) - 1
        self.low = self.low + int(range_size * symbol_index / total_symbols)
        self._normalize()

    def encode_escape(self, probability: float):
        range_size = self.high - self.low + 1
        self.high = self.low + int(range_size * probability) - 1
        self._normalize()

    def _normalize(self):
        while True:
            if (self.high ^ self.low) & 0x80000000 == 0:
                self._output_bit((self.high >> 31) & 1)
                self.low = (self.low << 1) & 0xFFFFFFFF
                self.high = ((self.high << 1) | 1) & 0xFFFFFFFF
            elif (self.low & 0x40000000) and not (self.high & 0x40000000):
                self.pending_bits += 1
                self.low = (self.low << 1) & 0x7FFFFFFF
                self.high = ((self.high << 1) | 0x80000001) & 0xFFFFFFFF
            else:
                break

    def _output_bit(self, bit: int):
        self.buffer = (self.buffer << 1) | bit
        self.buffer_pos += 1
        
        if self.buffer_pos == 8:
            self.output.append(self.buffer)
            self.buffer = 0
            self.buffer_pos = 0
        
        if self.pending_bits > 0:
            pending = self.pending_bits
            self.pending_bits = 0
            for _ in range(pending):
                self._output_bit(1 - bit)

    def finalize(self) -> bytes:
        # Добавляем завершающие биты
        self.pending_bits += 1
        bit = (self.low >> 30) & 1
        self._output_bit(bit)
        
        # Дополняем последний байт
        while self.buffer_pos > 0:
            self._output_bit(0)
            
        return bytes(self.output)

def ppmx_compress(data: Union[str, List[int]], max_order: int = 3) -> bytes:
    model = PPMModelX(max_order)
    encoder = ArithmeticEncoder1()
    context = []
    
    for symbol in data:
        encoded = False
        masked_symbols = set()
        
        # Пробуем кодировать от высшего порядка к низшему
        for order in range(min(len(context), max_order), -1, -1):
            current_context = tuple(context[-order:]) if order > 0 else tuple()
            
            if current_context in model.context_trees[order]:
                ctx = model.context_trees[order][current_context]
                
                if symbol in ctx['symbols']:
                    # Получаем список всех символов в контексте
                    symbol_list = list(ctx['symbols'].keys())
                    
                    # Кодируем символ
                    encoder.encode_symbol(symbol, symbol_list)
                    encoded = True
                    break
                else:
                    # Кодируем escape
                    p_escape = model.get_escape_probability(current_context)
                    encoder.encode_escape(p_escape)
                    masked_symbols.update(ctx['symbols'].keys())
        
        if not encoded:
            # Кодируем новый символ
            if not model.unique_symbols:
                # Первый символ кодируем как есть
                if isinstance(symbol, str):
                    encoder.output.extend(symbol.encode('utf-8'))
                else:
                    encoder.output.extend(symbol.to_bytes(4, 'big'))
            else:
                # Кодируем новый символ относительно известных
                symbol_list = list(model.unique_symbols)
                encoder.encode_symbol(symbol, symbol_list + [symbol])
        
        model.update_model(context, symbol)
        context.append(symbol)
        if len(context) > max_order:
            context.pop(0)
    
    return encoder.finalize()

   

In [32]:
with open("Results_text.txt", "w") as file:
    file.write('Результаты работы PPM\n')
m={}
for text in [text1, text2, text3, text4]:
    with open("Results_text.txt", "a") as file:
        file.write(f"\n\nИсходный текст: {text[:100]}...\nДлина: {len(text)} символов\nИсходный размер: {main_size} байт")
    m.clear()

    # Сжимаем с разными методами ухода
    for method in ['A', 'B', 'C', 'D', 'X']:
        # print(f"\nМетод ухода: {method}")
        if method=='X':
            # Сжатие
            compressed = ppmx_compress(text, max_order=5)
            # print(compressed)
            # print(f"Сжатые данные: {len(compressed)} байт ({len(compressed)/main_size*100:.2f}% от исходного)")
            m[method]= len(compressed)
            with open("Results_text.txt", "a") as file:
                file.write(f"\n\nМетод ухода: {method}\nСжатые данные: {len(compressed)} байт ({len(compressed)/main_size*100:.2f}% от исходного)")
    
        else:
            # Сжатие
            compressed = ppm_compress(text, max_order=5, escape_method=method)
            # print(compressed)
            # print(f"Сжатые данные: {len(compressed)} байт ({len(compressed)/main_size*100:.2f}% от исходного)")
            m[method]= len(compressed)
            with open("Results_text.txt", "a") as file:
                file.write(f"\n\nМетод ухода: {method}\nСжатые данные: {len(compressed)} байт ({len(compressed)/main_size*100:.2f}% от исходного)")
    
    with open("Results_text.txt", "a") as file:
        min_value = min(m.values())
        min_keys = [k for k, v in m.items() if v == min_value]
        if len(min_keys)==1:
            file.write(f"\n\nВывод по методу сжатия: Наилучший результат показал метод оценки вероятности ухода {min(m, key=m.get)} - {min_value} байт")
        else:
            file.write(f"\n\nВывод по методу сжатия: Наилучший результат показали методы оценки вероятности ухода {', '.join(map(str, min_keys))} - {min_value} байт")
        

        

Дополнительная кодировка массива исключения для декапитализированного текста

In [33]:
import re
def check_caps(t): # Поиск всех слов с заглавными буквами
    pattern = r'\b\w*[A-ZА-ЯЁ]\w*\b'
    # Поиск всех совпадений
    words_with_capital = re.findall(pattern, t)
    return words_with_capital

def cap_symb(t): # Поиск всех символов, являющихся заглавными буквами
    caps=0
    for char in t:
        if char.isupper():
            caps+=1
    return caps

In [34]:
def generate_bit_array(text, extra): # Применение общих правил декапитализации и создание битового массива, в котором 1 указывают на позиции исключений
    bit_array = []
    text_list = list(text)  # Преобразуем строку в список для изменения символов
    # print(text)
    expected_lowers=[True]*len(text)
    for i, char in enumerate(text):
        if char.isalpha():
            
            expected_lower = True  # по умолчанию ожидаем строчную
            
            if extra:
                # Проверка на название (2 заглавние подряд)
                if i>=2 and ((text[i - 2].isalpha() and text[i-2].isupper()) and (text[i-1].isalpha() and text[i-1].isupper())):
                    expected_lower = False
                    text_list[i]=text_list[i].lower()

            # Проверка правил
            if i == 0 or (i > 1 and text[i - 2:i] in {". ", "! ", "? ", "\n"}) or (i > 1 and text[i - 1:i] in {'"'}):
                expected_lower = False
                text_list[i]=text_list[i].lower()
                # caps.append(i)

            # Проверка местоимения 'I'
            if char == 'I' and (i == 0 or not text[i - 1].isalpha()) and (
                    i == len(text) - 1 or not text[i + 1].isalpha()):
                expected_lower = False
                text_list[i]=text_list[i].lower()
                # caps.append(i)

            # Проверка исключений
            if (char.islower() and not expected_lower) or (char.isupper() and expected_lower):
                bit_array.append(1)
                # caps.append(i)
            else:
                bit_array.append(0)

            expected_lowers[i]=expected_lower
        else:
            bit_array.append(0)  # не буквы игнорируем

    text_new = ''.join(text_list)
    return bit_array, text_new

In [35]:
rules, text_decap=generate_bit_array(text1, True)
print('Исходный размер массива исключений:')
# print(rules[:100])
size_of_exceptions=len(rules)/8 # в байтах
# print(len(rules))
bytes_info(size_of_exceptions)

# закодируем количество нулей между 1
numbers=[]
z=0
for i in rules:
    if i==0:
        z+=1
    else:
        numbers.append(z)
        z=0
print("Массив растояний между исключениями:", numbers)

intt=list(map(str, numbers))
string_int=','.join(intt)
# print(string_int)

Исходный размер массива исключений:
Размер файла: 396441.125 байт или  387.15 Кб или  0.378 Мб
Массив растояний между исключениями: [1, 7, 0, 6, 0, 3, 0, 6, 1, 0, 3, 0, 2, 0, 10, 3, 5, 8, 10, 8, 8, 10, 9, 5, 80, 535, 13, 323, 7, 215, 152, 19, 34, 147, 26, 670, 31, 17, 7, 6, 211, 91, 2, 2, 8, 24, 47, 65, 3, 187, 11, 82, 22, 17, 24, 30, 68, 9, 5, 94, 77, 24, 401, 96, 250, 41, 12, 117, 12, 21, 6, 47, 2, 2, 45, 31, 25, 97, 720, 193, 79, 265, 6, 0, 3, 3, 5, 0, 6, 3, 6, 23, 135, 8, 6, 9, 15, 10, 6, 7, 20, 9, 123, 137, 175, 8, 34, 14, 2, 5, 7, 10, 19, 8, 7, 10, 24, 9, 7, 11, 8, 5, 8, 7, 7, 5, 9, 7, 11, 9, 16, 8, 6, 8, 9, 20, 10, 13, 4, 7, 13, 50, 10, 118, 194, 178, 163, 128, 137, 0, 3, 0, 1, 0, 2, 0, 9, 0, 1, 0, 1, 0, 8, 0, 73, 56, 7, 12, 36, 32, 23, 19, 4, 32, 31, 4, 20, 56, 18, 36, 56, 57, 61, 104, 21, 6, 27, 29, 10, 16, 102, 58, 64, 112, 66, 61, 42, 545, 62, 80, 115, 122, 214, 331, 66, 24, 133, 5, 123, 12, 36, 14, 129, 131, 6, 53, 12, 88, 12, 160, 11, 74, 6, 50, 12, 41, 29, 89, 6, 232, 8, 

In [38]:
import struct
with open("Results_exceptions.txt", "w") as file:
    file.write('Результаты работы PPM\n')
m={}
data=len(struct.pack(f'{len(numbers)}q', *numbers))
with open("Results_exceptions.txt", "a") as file:
    file.write(f"\n\nИсходный текст: {numbers[:100]}...\nДлина: {len(numbers)} символов\nИсходный размер: {data} байт")
m.clear()

    # Сжимаем с разными методами ухода
for method in ['A', 'B', 'C', 'D', 'X']:    
    print(f"\nМетод ухода: {method}")
    if method=='X':
            # Сжатие
            compressed = ppmx_compress(numbers, max_order=5)
            print(compressed)
            print(f"Сжатые данные: {len(compressed)} байт ({len(compressed)/data*100:.2f}% от исходного)")
            m[method]= len(compressed)
            with open("Results_exceptions.txt", "a") as file:
                file.write(f"\n\nМетод ухода: {method}\nСжатые данные: {len(compressed)} байт ({len(compressed)/data*100:.2f}% от исходного)")
    else:
            # Сжатие
        compressed = ppm_compress(numbers, max_order=5, escape_method=method)
        print(compressed)
        print(f"Сжатые данные: {len(compressed)} байт ({len(compressed)/data*100:.2f}% от исходного)")
        m[method]= len(compressed)
        with open("Results_exceptions.txt", "a") as file:
            file.write(f"\n\nМетод ухода: {method}\nСжатые данные: {len(compressed)} байт ({len(compressed)/data*100:.2f}% от исходного)")
    
with open("Results_exceptions.txt", "a") as file:
    min_value = min(m.values())
    min_keys = [k for k, v in m.items() if v == min_value]
    if len(min_keys)==1:
        file.write(f"\n\nВывод по методу сжатия: Наилучший результат показал метод оценки вероятности ухода {min(m, key=m.get)} - {min_value} байт")
    else:
        file.write(f"\n\nВывод по методу сжатия: Наилучший результат показали методы оценки вероятности ухода {', '.join(map(str, min_keys))} - {min_value} байт")
        

        


Метод ухода: A
b'\x04\x00\x00\x00\x01@\x00\x00\x01\xc8\x00\x00\x00\x07\x80\x00\x00\x03I\x00\x00\x00\x01\xb2\x95X\x00\x00\x00\t\x0c\xe0\x00\x00\x02\xa4\xb6\x00\x00\x00\x15\x0c\x00\x00\x00C\xc5OP\x00\x00\x010\xd4\x00\x00\x00P\x1c\x00\x00\x02\x17[\x80\x00\x00\x06\x8b\xc0\x00\x00\xa1\x9eb\x00\x00\x01\xaev\x00\x00\x010\xe5\xc0\x00\x00\t\xa9\x00\x00\x00\x08\x9f\x80\x00\x00\x12dp\x00\x00\x01\xa4\x16\x00\x00\ny\xd6\x00\x00\x00\x1fz\xe0\x00\x00\x04S\xd7\x0e\x80\x00\x00\xd3i\x80\x00\x00\x16\xe1\x17\x9e\xd6\x00\x00\x00a\xd3\x80\x00\x00/s\x10\x00\x00\x08/\xc8@\x00\x00\x0b\xb7\xbf\x00\x00\x00\x16\xe0@\x00\x00\x14\x89@\x00\x00\x01f\xf1\xdah\x00\x00\x01\xe0\x84\x00\x00\x01\x11n\xb1\x80\x00\x00/*p\x00\x00\x04\xd7\xeb\xce\x00\x00\x06D\x17\xc0\x00\x0004\x90\x00\x00\x0f\xa3.\x00\x00\x00)b\xd0\x00\x00\x01\x8cf\x00\x00\x01\xd6\xf4\xb2\x80\x00\x00\x15}\xb2\xbc0\x04\x00\x00\x01k\xe5\x9a\x00\x00\x002OP\x00\x00\x0c#\xe4\x00\x00\x0bAj \x00\x000S4\x00\x00\x01=\xc0\xe0\x00\x00BL\x19q\x8c*\xa2\x9f\xce`\x00\x00\x0

In [39]:
with open("Results_text_decap.txt", "w") as file:
    file.write('Результаты работы PPM\n')
mm={}
for text in [text2, text4]:
    with open("Results_text_decap.txt", "a") as file:
        file.write(f"\n\nИсходный текст: {text[:100]}...\nДлина: {len(text)} символов\nИсходный размер: {main_size} байт")
    mm.clear()

    # Сжимаем с разными методами ухода
    for method in ['A', 'B', 'C', 'D', 'X']:
        # print(f"\nМетод ухода: {method}")
        if method=='X':
            # Сжатие
            compressed = ppmx_compress(text, max_order=5)
            # print(compressed)
            # print(f"Сжатые данные: {len(compressed)} байт ({len(compressed)/main_size*100:.2f}% от исходного)")
            m[method]= len(compressed)
            with open("Results_text_decap.txt", "a") as file:
                file.write(f"\n\nМетод ухода: {method}\nСжатые данные: {len(compressed)} байт ({len(compressed)/main_size*100:.2f}% от исходного)")
        else:
        # Сжатие
            compressed = ppm_compress(text, max_order=5, escape_method=method)
            # print(compressed)
            # print(f"Сжатые данные: {len(compressed)} байт ({len(compressed)/main_size*100:.2f}% от исходного)")
            mm[method]= len(compressed)
            size=mm[method]+m['X']
            with open("Results_text_decap.txt", "a") as file:
                file.write(f"\n\nМетод ухода: {method}\nСжатые данные: {size} байт ({size/main_size*100:.2f}% от исходного)")
    
    with open("Results_text_decap.txt", "a") as file:
        min_value = min(mm.values())
        min_keys = [k for k, v in mm.items() if v == min_value]
        if len(min_keys)==1:
            file.write(f"\n\nВывод по методу сжатия: Наилучший результат показал метод оценки вероятности ухода {min(mm, key=mm.get)} - {min_value+m['X']} байт")
        else:
            file.write(f"\n\nВывод по методу сжатия: Наилучший результат показали методы оценки вероятности ухода {', '.join(map(str, min_keys))} - {min_value+m['X']} байт")
        

        